# Accelerating End-to-End Data Science Workflows # 

## 04 - cuGraph ในฐานะแบ็คเอนด์ของ NetworkX  ##
**cuGraph as a NetworkX backend**

**สารบัญ**
<br>
สมุดบันทึก (notebook) นี้จะแนะนำวิธีการใช้งาน cuGraph backend สำหรับ NetworkX และรันอัลกอริทึม centrality กับชุดข้อมูล สมุดบันทึกนี้ครอบคลุมส่วนต่างๆ ดังนี้:
1. [ความเป็นมา](#Background)
2. [การติดตั้ง](#Installation)
3. [การใช้งาน nx-cugraph](#Utilizing-nx-cugraph)
    * [ตัวแปรสภาพแวดล้อมรันไทม์ (Runtime Environment Variable)](#Runtime-Environment-Variable)
    * [อาร์กิวเมนต์คีย์เวิร์ด Backend (Backend Keyword Argument)](#Backend-Keyword-Argument)
    * [การจัดส่งตามประเภท (Type-Based Dispatching)](#Type-Based-Dispatching)
4. [การคำนวณ Centrality](#Computing-Centrality)
    * [การสร้างกราฟ (Creating Graph)](#Creating-Graph)
    * [การรันอัลกอริทึม Centrality (Running Centrality Algorithms)](#Running-Centrality-Algorithms)
    * [Betweenness Centrality](#Betweenness-Centrality)
    * [Degree Centrality](#Degree-Centrality)
    * [Katz Centrality](#Katz-Centrality)
    * [Pagerank Centrality](#Pagerank-Centrality)
    * [Eigenvector Centrality](#Eigenvector-Centrality)
    * [การแสดงผลลัพธ์ (Visualize Results)](#Visualize-Results)
    * [แบบฝึกหัด #1 - การจัดส่งตามประเภท (Type Dispatch)](#Exercise-#1---Type-Dispatch)

## ความเป็นมา ##
RAPIDS เพิ่งเปิดตัวแบ็กเอนด์ใหม่สำหรับ NetworkX ที่เรียกว่า **nx-cugraph** ด้วยแบ็กเอนด์นี้ คุณสามารถเร่งความเร็วอัลกอริทึมที่รองรับได้โดยอัตโนมัติ ในหน้านี้ (notebook) เราจะกล่าวถึงวิธีการต่างๆ ในการเปิดใช้งานแบ็กเอนด์ cugraph และใช้แบ็กเอนด์นี้เพื่อเรียกใช้อัลกอริทึม centrality ที่แตกต่างกัน

## การติดตั้ง ##
เราได้เตรียมสภาพแวดล้อมโดยติดตั้ง **nx-cugraph** ไว้เรียบร้อยแล้ว หากคุณใช้งานในสภาพแวดล้อมของคุณเอง คำสั่งสำหรับติดตั้งจะอยู่ด้านล่างนี้

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## การใช้งาน nx-cugraph ##
มี 3 วิธีในการใช้งาน nx-cugraph:

1.  **Environment Variable at Runtime**
2.  **Backend keyword argument**
3.  **Type-Based dispatching**

เรามาเจาะลึกแต่ละวิธีกันเลย


### ตัวแปรสภาพแวดล้อมรันไทม์ ###
สามารถใช้ตัวแปรสภาพแวดล้อม `NETWORKX_AUTOMATIC_BACKENDS` เพื่อให้ NetworkX เลือกใช้แบ็กเอนด์ที่ระบุโดยอัตโนมัติได้ ตั้งค่า `NETWORKX_AUTOMATIC_BACKENDS=cugraph` เพื่อใช้ `nx-cugraph` ในการเร่งความเร็ว API ที่รองรับด้วย GPU โดยไม่ต้องแก้ไขโค้ด เราจะโหลดโมดูล cuDF pandas เพื่อช่วยเร่งความเร็วในการโหลดไฟล์ CSV ด้วย

In [ ]:
!NETWORKX_AUTOMATIC_BACKENDS=cugraph python -m cudf.pandas scripts/networkx.py

### อาร์กิวเมนต์คีย์เวิร์ด Backend ###
NetworkX ยังรองรับการระบุ **backend (แบ็กเอนด์)** ที่เฉพาะเจาะจงสำหรับ API ที่รองรับด้วยอาร์กิวเมนต์ **`backend=`** อาร์กิวเมนต์นี้จะมีลำดับความสำคัญเหนือกว่าตัวแปรสภาพแวดล้อม `NETWORKX_AUTOMATIC_BACKENDS` วิธีนี้ยังกำหนดให้แบ็กเอนด์ที่ระบุต้องติดตั้งไว้แล้วด้วย

In [ ]:
import warnings
warnings.filterwarnings('ignore')

%load_ext cudf.pandas
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt

# Load the CSV file
road_graph = pd.read_csv('./data/road_graph.csv', dtype=['int32', 'int32', 'float32'], nrows=1000)

# Create an empty graph
G = nx.from_pandas_edgelist(road_graph, source='src', target='dst', edge_attr='length')
b = nx.betweenness_centrality(G, k=1000, backend="cugraph")

### การจัดส่งตามประเภท (Type-Based Dispatching) ###
สำหรับผู้ใช้ที่ต้องการให้แน่ใจว่าการทำงานเป็นไปตามที่ต้องการ โดยไม่มีการแปลงประเภทข้อมูลในขณะรันไทม์ (runtime conversions) นั้น **NetworkX มีการเลือกฟังก์ชันตามประเภท (type-based dispatching)** เพื่อใช้งานวิธีนี้ ผู้ใช้จะต้องนำเข้า (import) แบ็กเอนด์ (backend) ที่ต้องการ และสร้างอินสแตนซ์ของกราฟ (Graph instance) สำหรับแบ็กเอนด์นั้น

In [ ]:
import networkx as nx
import nx_cugraph as nxcg

# Loading data from previous cell
G = nx.from_pandas_edgelist(road_graph, source='src', target='dst', edge_attr='length') 

nxcg_G = nxcg.from_networkx(G)             # conversion happens once here
b = nx.betweenness_centrality(nxcg_G, k=1000)  # nxcg Graph type causes cugraph backend to be used, no conversion necessary

## การคำนวณ Centrality ##
ตอนนี้เราได้เรียนรู้วิธีการเปิดใช้งาน nx-cugraph แล้ว มาลองใช้มันในเวิร์กโฟลว์กันเถอะ! เราจะใช้ `backend` argument สำหรับตัวอย่างนี้ ขั้นแรก มาสร้างกราฟกันก่อน

### การสร้างกราฟ (Creating Graph) ###

In [ ]:
# Create a graph from already loaded dataframe
G = nx.from_pandas_edgelist(road_graph, source='src', target='dst', edge_attr='length')

### การรันอัลกอริทึม Centrality ###
ตอนนี้ เรามาลองรันอัลกอริทึม centrality ต่างๆ กันเลย!

### Betweenness Centrality ###
เป็นการหาปริมาณจำนวนครั้งที่ **จุดเชื่อม (node)** ทำหน้าที่เป็น **สะพาน (bridge)** บนเส้นทางที่สั้นที่สุดระหว่างจุดเชื่อมอื่นสองจุด ซึ่งจะเน้นย้ำถึงความสำคัญของจุดเชื่อมนั้นในการไหลของข้อมูล

In [ ]:
b = nx.betweenness_centrality(G, backend="cugraph")

### Degree Centrality ###
วัดจำนวนการเชื่อมต่อโดยตรงที่ **จุดเชื่อม (node)** หนึ่งๆ มี ซึ่งบ่งชี้ว่าจุดนั้นมีการเชื่อมโยงที่ดีเพียงใดภายในเครือข่าย

In [ ]:
d = nx.degree_centrality(G, backend="cugraph")

### Katz Centrality ###
เป็นการวัด **ความเป็นศูนย์กลาง (centrality)** ของจุดเชื่อม (node) โดยพิจารณาจาก **อิทธิพลโดยรวม (global influence)** ในเครือข่าย ทั้งการเชื่อมต่อโดยตรงและโดยอ้อม

In [ ]:
k = nx.katz_centrality(G, backend="cugraph")

### Pagerank Centrality ###
พิจารณาความสำคัญของ **โหนด (node)** โดยอิงตามปริมาณและคุณภาพของลิงก์ที่เชื่อมโยงมายังโหนดนั้น คล้ายคลึงกับอัลกอริทึม PageRank ดั้งเดิมของ Google

In [ ]:
p = nx.pagerank(G, max_iter=10, tol=1.0e-3, backend="cugraph")

### Eigenvector Centrality ###
ให้คะแนน **จุดเชื่อม (nodes)** โดยยึดหลักว่า การเชื่อมต่อกับ **จุดเชื่อมที่ได้คะแนนสูง (high-scoring nodes)** จะมีส่วนช่วยเพิ่มคะแนนของจุดเชื่อมนั้นๆ ได้มากกว่าการเชื่อมต่อกับ **จุดเชื่อมที่ได้คะแนนต่ำ (low-scoring nodes)**

In [ ]:
e = nx.eigenvector_centrality(G, max_iter=1000, tol=1.0e-3, backend="cugraph")

### การแสดงผลลัพธ์ ###
ตอนนี้เรามาลองดูผลลัพธ์กัน! เราจะแสดงแค่ 5 แถวแรกเพื่อให้ง่ายต่อการอ่าน

In [ ]:
from IPython.display import display_html
dc_top = pd.DataFrame(sorted(d.items(), key=lambda x:x[1], reverse=True)[:5], columns=["vertex", "degree_centrality"])
bc_top = pd.DataFrame(sorted(b.items(), key=lambda x:x[1], reverse=True)[:5], columns=["vertex", "betweenness_centrality"])
katz_top = pd.DataFrame(sorted(k.items(), key=lambda x:x[1], reverse=True)[:5], columns=["vertex", "katz_centrality"])
pr_top = pd.DataFrame(sorted(p.items(), key=lambda x:x[1], reverse=True)[:5], columns=["vertex", "pagerank"])
ev_top = pd.DataFrame(sorted(e.items(), key=lambda x:x[1], reverse=True)[:5], columns=["vertex", "eigenvector_centrality"])

df1_styler = dc_top.style.set_table_attributes("style='display:inline'").set_caption('Degree').hide(axis='index')
df2_styler = bc_top.style.set_table_attributes("style='display:inline'").set_caption('Betweenness').hide(axis='index')
df3_styler = katz_top.style.set_table_attributes("style='display:inline'").set_caption('Katz').hide(axis='index')
df4_styler = pr_top.style.set_table_attributes("style='display:inline'").set_caption('PageRank').hide(axis='index')
df5_styler = ev_top.style.set_table_attributes("style='display:inline'").set_caption('EigenVector').hide(axis='index')

display_html(df1_styler._repr_html_()+df2_styler._repr_html_()+df3_styler._repr_html_()+df4_styler._repr_html_()+df5_styler._repr_html_(), raw=True)

### แบบฝึกหัด #1 - การจัดส่งตามประเภท (Type Dispatch) ###
ใช้ **type dispatching method** เพื่อให้ได้ผลลัพธ์ **Pagerank centrality** โดยใช้ **cugraph backend**

Click ... for solution. 

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

**เยี่ยมมาก!**